# Ring Algorithm using PGX

## First Step: Get a session in PGX server.

In [1]:
import json
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

def generateToken(base_url, username, password):
    from urllib.request import Request, urlopen
    from urllib.error import HTTPError

    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)

    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')

    except HTTPError as err:

        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))

        os._exit(1)

baseUrl = 'https://localhost:7007'
username = 'workshop022'
password = 'welcome1'

token=generateToken(baseUrl, username, password)

In [2]:
print(token)

eyJraWQiOiJEYXRhYmFzZVJlYWxtIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJXT1JLU0hPUDAyMiIsInJvbGVzIjpbIkdSQVBIX0RFVkVMT1BFUiIsIkdSQVBIX0FETUlOSVNUUkFUT1IiXSwiZ2VuZXJhbF9wZXJtaXNzaW9ucyI6WyJQR1hfU0VTU0lPTl9HRVRfUFVCTElTSEVEX0dSQVBIIiwiUEdYX1NFUlZFUl9HRVRfSU5GTyIsIlBHWF9TRVNTSU9OX01PRElGWV9NT0RFTCIsIlBHWF9TRVNTSU9OX1JFQURfTU9ERUwiLCJQR1hfU0VTU0lPTl9DUkVBVEUiLCJQR1hfU0VSVkVSX01BTkFHRSIsIlBHWF9TRVNTSU9OX05FV19HUkFQSCJdLCJpc3MiOiJvcmFjbGUucGcuaWRlbnRpdHkucmVzdC5BdXRoZW50aWNhdGlvblNlcnZpY2UiLCJleHAiOjE2MTYzOTk3NzgsIm51bV90b2tlbl9yZWZyZXNoIjowLCJmaWxlX3Blcm1pc3Npb25zIjp7IlwvaG9tZVwvb3JhY2xlXC9Eb3dubG9hZHNcL2JzYW4iOlsiUkVBRCJdfX0.pM-Ne-_0D2_xEf_wpOMCNFpZ96igonTCeJwBGM3VCdPuvSxgjXR5wtTBUaEcLEX5ZqYLoe4zWHNiVOYnFlrAKH298AkQKfm1Lcclejc5ANvsSS6rvxDmbnr9KG7ZGsMsC3pSopv6p9AhjUbOrJmmosIkVUD7XSiWQiwO04xn0_qJ4IyCib2Mqb_n-uYKJfSerd67nXBF2JFoA1a41Ql-kvrlwjzAhWpfgzWAH3EBQyFz-sxNt5MbBid8pLxfX3RNQR-d_GOiUwhKudiU731isR9ILRasdRUd7JkQNk9TooA5vuDomn40IIGYAF9w1QviUbq_fJCVcxPP7fug6_D6mw


In [3]:
try:
    import pypgx as pgx

    session = pgx.get_session(base_url=baseUrl, token=token)
    instance = session.server_instance
    analyst = session.create_analyst()

except SystemExit:
    os._exit(0)
    

In [4]:
print(session)

PgxSession(id: 1a5cadf9-2b36-4729-a6d2-4e10f412e6b1, name: pypgx)


## Load JSON File with Property Graph Definition

In [6]:
# read Graph
graph = session.read_graph_with_properties("./bank.json")


RuntimeError: JVM exception occurred: java.lang.Exception: ORA-00942: table or view does not exist
 code: PGX-ERROR-2R8OO37VPPJTC java.util.concurrent.ExecutionException
Java stack trace:
    java.util.concurrent.ExecutionException: java.lang.Exception: ORA-00942: table or view does not exist
 code: PGX-ERROR-2R8OO37VPPJTC
    java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:357)
    java.util.concurrent.CompletableFuture.get(CompletableFuture.java:1908)
    oracle.pgx.api.PgxFuture.get(PgxFuture.java:99)
    oracle.pgx.api.PgxSession.readGraphWithProperties(PgxSession.java:1959)
Caused by:
    java.lang.Exception: ORA-00942: table or view does not exist
 code: PGX-ERROR-2R8OO37VPPJTC
    oracle.pgx.common.marshalers.ExceptionMarshaler.toUnserializedException(ExceptionMarshaler.java:110)
    oracle.pgx.common.marshalers.ExceptionMarshaler.unmarshal(ExceptionMarshaler.java:123)
    oracle.pgx.client.RemoteUtils.parseExceptionalResponse(RemoteUtils.java:130)
    or

In [ ]:
print('Edges number: ' + str(graph.num_edges) + ' Nodes number: '+ str(graph.num_vertices))

## Entity Link Analysis

### Performing entity link analysis on the above Graph data model is demonstrated below. We use brackets in the below table is to isolate individual elements of a collection.

### Find account who share more than one piece of legitimate contact

In [ ]:
pgxResultSetNode = graph.query_pgql("""
  select label(contact), listagg(acct.FIRSTNAME,','), count(*) as size
from match (acct)-[]->(contact)
where exists (
  SELECT count(*) as ringsize, contact
  from MATCH (account:CUST)-[]->(contact)
  group by contact 
  having ringsize > 1 
  order by ringsize desc
)
group by label(contact)
""")

for i in pgxResultSetNode:
    print (i)

### Determine the financial risk of a possible fraud ring

In [ ]:
pgxResultSetNode = graph.query_pgql("""
select label(contact), listagg(acct.FIRSTNAME,','), 
        count(*) as size, round(sum(CASE 
                                     WHEN label(r)='HAS_CREDITCARDS' THEN unsecuredAccount.LIMIT 
                                     WHEN label(r)='HAS_UNSECUREDLOANS' THEN unsecuredAccount.BALANCE 
                                     ELSE 0 
                                    END)) as FINANCIALRISK
from match (acct)-[]->(contact), 
     match (acct)-[r:HAS_CREDITCARDS|HAS_UNSECUREDLOANS]->(unsecuredAccount)
where exists (
  SELECT count(*) as ringsize, contact
  from MATCH (account:CUST)-[]->(contact)
  group by contact 
  having ringsize > 1 
  order by ringsize desc
)
group by label(contact)
""")

for i in pgxResultSetNode:
    print (i)

## Destroy Graph In PGX Memory

In [ ]:
session.destroy()